In [1]:
import pandas as pd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from geojson import Point, Feature, FeatureCollection, dump
import requests
import json
import math
import numpy
import os
import sys
import datetime
import numpy as np

In [2]:
FSR = pd.read_csv('./311data_2022/FSR_2022.csv')
FI = pd.read_csv('./311data_2022/FI_2022.csv')
FWO = pd.read_csv('./311data_2022/FWO_2022.csv')
FRA = pd.read_csv('./311data_2022/FRA_2022.csv')

len(FSR), len(FI), len(FWO), len(FRA)

/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_26505/8784320.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  FSR = pd.read_csv('./311data_2022/FSR_2022.csv')
/var/folders/sw/bm9l8xdn4yxdqvn80kxmq7y00000gn/T/ipykernel_26505/8784320.py:3: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  FWO = pd.read_csv('./311data_2022/FWO_2022.csv')


(92360, 172975, 136677, 174537)

In [3]:
# FSR.head()["createddate"]
FSR.columns

Index(['OBJECTID', 'ComplaintNumber', 'SRCategory', 'SRType', 'SRPriority',
       'SRSource', 'SRStatus', 'SRResolution', 'BoroughCode', 'CommunityBoard',
       'BuildingNumber', 'StreetName', 'CrossStreet1', 'CrossStreet2',
       'TaxClass', 'LocationDetails', 'ComplaintDetails', 'ParkName',
       'ParkZone', 'NotesToCustomer', 'Postcode', 'Council District ',
       'StateSenate', 'StateAssembly', 'Congressional', 'SanitationZone',
       'PhysicalID', 'ServiceRequestParentGlobalID', 'Geometry', 'GlobalID',
       'InitiatedDate', 'ClosedDate', 'CreatedDate', 'UpdatedDate',
       'Descriptor1', 'ComplaintType', 'CallerZipCode', 'SRCallerType',
       'CallerCity', 'CallerState', 'Latitude', 'Longitude', 'Census Tract',
       'BIN', 'BBL', 'NTA', 'Location', 'Zip Codes 2', 'Community Districts 2',
       'Borough Boundaries 2', 'City Council Districts 2',
       'Police Precincts 2'],
      dtype='object')

In [4]:
FI.columns

Index(['OBJECTID', 'InspectionType', 'InspectionStatus',
       'InspectionTPCondition', 'InspectionTPStructure', 'TreePointDBH',
       'SWTreeConditionRating', 'SWLocationRating', 'SWClearanceRating',
       'SWDamageRating', 'SWDamagedFlagstonesRating', 'SWRatingTotal',
       'TreePointGlobalID', 'PlantingSpaceGlobalID', 'ServiceRequestGlobalID',
       'Geometry', 'GlobalID', 'InspectionDate', 'ClosedDate', 'CreatedDate',
       'UpdatedDate', 'ParentInspectionGlobalID', 'ReinspectionDate',
       'SWTotalTrees', 'Location', 'Zip Codes 2', 'Community Districts 2',
       'Borough Boundaries 2', 'City Council Districts 2',
       'Police Precincts 2'],
      dtype='object')

In [5]:
FWO.columns

Index(['OBJECTID', 'WOType', 'WOStatus', 'WOPriority', 'Borough',
       'CommunityBoard', 'BuildingNumber', 'StreetName', 'LocationDetails',
       'ActualFinishDate', 'WOContract', 'ParkName', 'ParkZone',
       'CancelReason', 'WOCategory', 'SanitationAssignedDate',
       'SanitationRemovalDate', 'SanitationUpdatedDate', 'WOWoodRemains',
       'WOEquipment', 'Postcode', 'CityCouncil', 'StateSenate',
       'StateAssembly', 'Congressional', 'SanitationZone', 'PhysicalID',
       'CrossStreet1', 'CrossStreet2', 'PlantingSpaceGlobalID',
       'TreePointGlobalID', 'InspectionGlobalID', 'GlobalID', 'SidewalkDamage',
       'WOWireConflict', 'CrewGlobalID', 'Geometry', 'ClosedDate',
       'CancelDate', 'CreatedDate', 'UpdatedDate', 'WOEntity', 'PROJSTARTDATE',
       'WOProject', 'RecommendedSpecies', 'Latitude', 'Longitude',
       'Census Tract', 'BIN', 'BBL', 'NTA', 'Location', 'Zip Codes 2',
       'Community Districts 2', 'Borough Boundaries 2',
       'City Council Districts 2',

In [6]:
FRA.columns

Index(['OBJECTID', 'RADefect', 'RADefectLocation', 'Failure', 'ImpactTarget',
       'Consequence', 'RiskRating', 'InspectionGlobalID', 'GlobalID',
       'CreatedDate', 'FailureImpact', 'WorkOrderGlobalID'],
      dtype='object')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
# FSR Global ID corresponds to FI Service Request Global ID
len(set(FI.ServiceRequestGlobalID) - set(FSR.GlobalID)) 

4837

In [5]:
# FI GLOBAL ID corresponds to FWO Inspection Global ID
len(set(FWO.InspectionGlobalID) - set(FI.GlobalID))

1461

In [6]:
# FRA IncidentGlobalID corresponds to FI Global ID, roughly...
len(set(FRA.InspectionGlobalID) - set(FI.GlobalID))

1543

In [7]:
FSR_columns = ['SRCategory', 'SRType', 'SRPriority',
       'SRSource', 'SRStatus', 'SRResolution', 'BoroughCode', 'CommunityBoard',
       'ServiceRequestParentGlobalID', 'GlobalID',
       'InitiatedDate', 'ClosedDate', 'CreatedDate', 'UpdatedDate',
       'Descriptor1', 'ComplaintType', 'CallerZipCode', 'SRCallerType',
       'Latitude', 'Longitude', 'Census Tract', 'NTA', 'Zip Codes 2']

FI_columns = ['InspectionType', 'InspectionStatus',
       'InspectionTPCondition', 'InspectionTPStructure', 'TreePointDBH',
       'GlobalID', 'ServiceRequestGlobalID', 'InspectionDate', 'ClosedDate', 'CreatedDate',
       'UpdatedDate', 'ParentInspectionGlobalID', 'ReinspectionDate','Location']

FWO_columns = ['WOType', 'WOStatus', 'WOPriority', 'Borough',
       'InspectionGlobalID', 'GlobalID', 'ClosedDate',
       'CancelDate', 'CancelReason','CreatedDate', 'UpdatedDate', 'WOEntity', 
       'PROJSTARTDATE', 'WOProject', 'WOCategory', 'RecommendedSpecies', 'Location']

FRA_columns = ['RADefect', 'RADefectLocation', 'Failure', 'ImpactTarget',
       'Consequence', 'RiskRating', 'InspectionGlobalID', 'GlobalID',
       'CreatedDate', 'FailureImpact', 'WorkOrderGlobalID']

# Deal with empty return

In [ ]:
# Unlikely needed as we are processing all past dataset

In [ ]:
# if (len(FSR) == 0):
#     FSR = pd.DataFrame(columns=FSR_columns)

In [ ]:
# if (len(FI) == 0):
#     FI = pd.DataFrame(columns=FI_columns)

In [ ]:
# if (len(FWO) == 0):
#     FWO = pd.DataFrame(columns=FWO_columns)

In [ ]:
# if (len(FRA) == 0):
#     FRA = pd.DataFrame(columns=FRA_columns)

# Only keep subset of data

In [ ]:
# insert columns if not exits, unnecessary if using a large csv dataset
# FSR = FSR.reindex(columns=FSR_columns)
# FI = FI.reindex(columns=FI_columns)
# FWO = FI.reindex(columns=FWO_columns).fillna('N/A')
# FRA = FRA.reindex(columns=FRA_columns).fillna('N/A')

In [8]:
# select subset due to memory limit
FSR = FSR[FSR_columns]
FI = FI[FI_columns]
FWO = FWO[FWO_columns]
FRA = FRA[FRA_columns]

In [10]:
# set the globalid to its parent ID if the parentId is not NaN
FSR.loc[:, "GLOBAL_ID"] = FSR.apply(lambda x:  x.GlobalID if x.ServiceRequestParentGlobalID != x.ServiceRequestParentGlobalID else x.ServiceRequestParentGlobalID, axis = 1)

In [11]:
FSR.rename(columns={'GLOBAL_ID': 'SRGlobalID', 'ClosedDate': 'SRClosedDate', 'CreatedDate': 'SRCreatedDate', 'UpdatedDate': 'SRUpdatedDate'}, inplace=True)
FI.rename(columns={'GlobalID': 'InsGlobalID', 'ClosedDate': 'InsClosedDate', 'CreatedDate': 'InsCreatedDate', 'UpdatedDate': 'InsUpdatedDate', 'ServiceRequestGlobalID': 'SRGlobalID'}, inplace=True)
FWO.rename(columns={'GlobalID': 'WOGlobalID', 'ClosedDate': 'WOClosedDate', 'CreatedDate': 'WOCreatedDate', 'UpdatedDate': 'WOUpdatedDate', 'InspectionGlobalID': 'InsGlobalID'}, inplace=True)
FRA.rename(columns={'GlobalID': 'RAGlobalID', 'CreatedDate': 'RACreatedDate', 'InspectionGlobalID': 'InsGlobalID'}, inplace=True)

# not further process inspection dataset because only one inspection is needed for under-reporting measuring, but more inspections could still be used for other purposes

In [12]:
FSR['Longitude'] = FSR['Longitude'].astype(float)
FSR['Latitude'] = FSR['Latitude'].astype(float)

In [24]:
#requests that have no geo-info
# FSR.Longitude.isna().sum(), FSR.Latitude.isna().sum() 

(7369, 7369)

In [ ]:
# # find census tract
# geometry = [None if math.isnan(xy[0]) or math.isnan(xy[1]) else Point(xy) for xy in zip(FSR.longitude, FSR.latitude)]
# gdf = gpd.GeoDataFrame(FSR, crs=4326, geometry=geometry)

# Merge Inspections

In [13]:
FI.columns

Index(['InspectionType', 'InspectionStatus', 'InspectionTPCondition',
       'InspectionTPStructure', 'TreePointDBH', 'InsGlobalID', 'SRGlobalID',
       'InspectionDate', 'InsClosedDate', 'InsCreatedDate', 'InsUpdatedDate',
       'ParentInspectionGlobalID', 'ReinspectionDate', 'Location'],
      dtype='object')

In [44]:
# FI["Longitude"] = FI.Location.copy().apply(lambda x: float(x[0])).copy()
# FI["Latitude"] = FI.Location.copy().apply(lambda x: float(x[1])).copy()

In [ ]:
# geometry = [None if math.isnan(xy[0]) or math.isnan(xy[1]) else Point(xy) for xy in zip(FI.longitude, FI.latitude)]
# g_FI = gpd.GeoDataFrame(FI, crs=4326, geometry=geometry)

In [ ]:
# back to regular df
# merge_FI = pd.DataFrame(g_FI)

In [14]:
# merge service requests w/ inspections
# Q: if a service request has a parent request id, is SRGlobalId in inspection dataset associated with serviceID or its parentID?
mergeddf = pd.merge(FSR, FI, on='SRGlobalID', how='left', suffixes=('_SR', '_I'), left_index=False, right_index=False)

In [15]:
#Inspections with SR
a = FI[FI["SRGlobalID"].notna()]
len(a)

##There could be many service requests with no inspections

34791

In [16]:
mergeddf.columns

Index(['SRCategory', 'SRType', 'SRPriority', 'SRSource', 'SRStatus',
       'SRResolution', 'BoroughCode', 'CommunityBoard',
       'ServiceRequestParentGlobalID', 'GlobalID', 'InitiatedDate',
       'SRClosedDate', 'SRCreatedDate', 'SRUpdatedDate', 'Descriptor1',
       'ComplaintType', 'CallerZipCode', 'SRCallerType', 'Latitude',
       'Longitude', 'Census Tract', 'NTA', 'Zip Codes 2', 'SRGlobalID',
       'InspectionType', 'InspectionStatus', 'InspectionTPCondition',
       'InspectionTPStructure', 'TreePointDBH', 'InsGlobalID',
       'InspectionDate', 'InsClosedDate', 'InsCreatedDate', 'InsUpdatedDate',
       'ParentInspectionGlobalID', 'ReinspectionDate', 'Location'],
      dtype='object')

In [17]:
FSR.shape, FI.shape, mergeddf.shape 
# since merged > FSR, means there are some SRs with multiple inspections
# FI.length > FSR.length, there could be inspections without service requests (no global SRID). 

((92360, 24), (172975, 14), (96297, 37))

In [21]:
#Service requests with no inspections
len(mergeddf[mergeddf['InsGlobalID'].isna()])

53074

In [19]:
mergeddf.head()

,SRCategory,SRType,SRPriority,SRSource,SRStatus,SRResolution,BoroughCode,CommunityBoard,ServiceRequestParentGlobalID,GlobalID,InitiatedDate,SRClosedDate,SRCreatedDate,SRUpdatedDate,Descriptor1,ComplaintType,CallerZipCode,SRCallerType,Latitude,Longitude,Census Tract,NTA,Zip Codes 2,SRGlobalID,InspectionType,InspectionStatus,InspectionTPCondition,InspectionTPStructure,TreePointDBH,InsGlobalID,InspectionDate,InsClosedDate,InsCreatedDate,InsUpdatedDate,ParentInspectionGlobalID,ReinspectionDate,Location
0,Plant Tree,Street Tree,Routine,3-1-1 Call Center,Open,Reviewed - Inspection Assigned,Queens,411.0,NaN,3EC81228-3DE6-4314-8339-F89A0720A9A8,2020-11-05 18:09:20.0000000,NaN,2022-07-07 15:28:00.0000000,2022-10-03 13:03:00.0000000,For One Address,New Tree Request,NaN,Business\n\nBusiness,40.766138,-73.745175,1479.00,Douglaston-Little Neck,14505.0,3EC81228-3DE6-4314-8339-F89A0720A9A8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Prune,Clear Building,Routine,3-1-1 Call Center,Closed,Inspected - No Work Order,Bronx,209.0,NaN,4B8D16C7-CA9C-49A8-9219-172667426BEA,2021-01-27 14:18:08.0000000,2022-09-09 13:36:28.0000000,2022-07-07 15:30:53.0000000,2022-09-09 13:36:28.0000000,Hitting Building,Overgrown Tree/Branches,NaN,NaN,40.826875,-73.846715,98.00,Castle Hill-Unionport,11611.0,4B8D16C7-CA9C-49A8-9219-172667426BEA,Level 2,Closed,Good,Full,14.0,57CF781A-0975-4BFE-9904-D6CD8A6EE5A3,2022-09-09 13:36:22.0000000,2022-09-09 13:36:22.0000000,2022-09-09 13:36:22.0000000,NaN,NaN,NaN,POINT (-73.84681453739462 40.826964362109955)
2,Plant Tree,Street Tree,Routine,3-1-1 Call Center,Open,Reviewed - Inspection Assigned,Staten Island,502.0,NaN,77D798B5-4687-4430-A2AE-AE693E674AB0,2021-04-13 18:09:06.0000000,NaN,2022-07-07 15:28:00.0000000,2022-10-03 13:04:00.0000000,For One Address,New Tree Request,NaN,Business\n\nBusiness,40.599059,-74.126842,187.03,Todt Hill-Emerson Hill-Lighthouse Hill-Manor H...,10700.0,77D798B5-4687-4430-A2AE-AE693E674AB0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hazard,Hanging Limb Other,Routine,3-1-1 Call Center,Open,Work Priority Category - C,Queens,407.0,799570A8-71A4-4361-A6A8-464FA8AD9448,591583A7-B107-49EC-BFA7-415BE6D3EA4B,2021-07-31 20:32:57.0000000,NaN,2022-07-07 15:30:49.0000000,2022-07-08 14:13:46.0000000,Branch Cracked and Will Fall,Damaged Tree,NaN,NaN,40.769165,-73.800688,1147.00,Murray Hill-Broadway Flushing,14193.0,799570A8-71A4-4361-A6A8-464FA8AD9448,Level 2,Closed,Good,Full,33.0,BD916064-C30D-48F7-9397-F66DCF01203D,2022-06-13 04:00:00.0000000,2022-06-13 19:12:00.0000000,2022-06-13 19:09:00.0000000,2022-06-13 19:15:11.0000000,NaN,NaN,POINT (-73.80039800602839 40.7690878004193)
4,Root/Sewer/Sidewalk,Trees and Sidewalks,Routine,3-1-1 Call Center,Closed,T/S Inspected - Repair Pending,Brooklyn,317.0,NaN,3831B9B0-6787-4A4B-90EF-BC83CDB67735,2021-10-14 22:50:02.0000000,2022-08-05 15:40:29.0000000,2022-07-07 15:30:52.0000000,2022-08-05 15:40:29.0000000,Trees and Sidewalks Program,Root/Sewer/Sidewalk Condition,NaN,NaN,40.638333,-73.934780,836.00,East Flatbush-Farragut,16866.0,3831B9B0-6787-4A4B-90EF-BC83CDB67735,Sidewalk Rating,Closed,Good,Full,22.0,1D67BA0C-DDA6-40F2-B4BA-89F6D185AEF6,2022-08-05 15:40:07.0000000,2022-08-05 15:40:07.0000000,2022-08-05 15:40:07.0000000,NaN,NaN,NaN,POINT (-73.93487015325061 40.63845723855442)


In [22]:
#fill empty inspection ID 
mergeddf['InsGlobalID'].fillna('-', inplace=True)

# Merge Work Orders

In [23]:
FWO.columns

Index(['WOType', 'WOStatus', 'WOPriority', 'Borough', 'InsGlobalID',
       'WOGlobalID', 'WOClosedDate', 'CancelDate', 'CancelReason',
       'WOCreatedDate', 'WOUpdatedDate', 'WOEntity', 'PROJSTARTDATE',
       'WOProject', 'WOCategory', 'RecommendedSpecies', 'Location'],
      dtype='object')

In [ ]:
# FWO["longitude"] = FWO.location.copy().apply(lambda x: float(x['coordinates'][0])).copy()
# FWO["latitude"] = FWO.location.copy().apply(lambda x: float(x['coordinates'][1])).copy()

# geometry = [None if math.isnan(xy[0]) or math.isnan(xy[1]) else Point(xy) for xy in zip(FWO.longitude, FWO.latitude)]
# g_FWO = gpd.GeoDataFrame(FWO, crs=4326, geometry=geometry)

In [ ]:
# back to regular df
# merge_FWO = pd.DataFrame(g_FWO)

In [24]:
# merge SRs+Insps w/ WOs
mergeddf = pd.merge(mergeddf, FWO, on='InsGlobalID', how='left', suffixes=('', '_WO'))

((136677, 17), (96368, 53))

In [25]:
FWO.shape, mergeddf.shape #FWO length > mergeddf length, meaning not all work orders have inspections

((136677, 17), (96368, 53))

In [26]:
mergeddf.head()

,SRCategory,SRType,SRPriority,SRSource,SRStatus,SRResolution,BoroughCode,CommunityBoard,ServiceRequestParentGlobalID,GlobalID,InitiatedDate,SRClosedDate,SRCreatedDate,SRUpdatedDate,Descriptor1,ComplaintType,CallerZipCode,SRCallerType,Latitude,Longitude,Census Tract,NTA,Zip Codes 2,SRGlobalID,InspectionType,InspectionStatus,InspectionTPCondition,InspectionTPStructure,TreePointDBH,InsGlobalID,InspectionDate,InsClosedDate,InsCreatedDate,InsUpdatedDate,ParentInspectionGlobalID,ReinspectionDate,Location,WOType,WOStatus,WOPriority,Borough,WOGlobalID,WOClosedDate,CancelDate,CancelReason,WOCreatedDate,WOUpdatedDate,WOEntity,PROJSTARTDATE,WOProject,WOCategory,RecommendedSpecies,Location_WO
0,Plant Tree,Street Tree,Routine,3-1-1 Call Center,Open,Reviewed - Inspection Assigned,Queens,411.0,NaN,3EC81228-3DE6-4314-8339-F89A0720A9A8,2020-11-05 18:09:20.0000000,NaN,2022-07-07 15:28:00.0000000,2022-10-03 13:03:00.0000000,For One Address,New Tree Request,NaN,Business\n\nBusiness,40.766138,-73.745175,1479.00,Douglaston-Little Neck,14505.0,3EC81228-3DE6-4314-8339-F89A0720A9A8,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Prune,Clear Building,Routine,3-1-1 Call Center,Closed,Inspected - No Work Order,Bronx,209.0,NaN,4B8D16C7-CA9C-49A8-9219-172667426BEA,2021-01-27 14:18:08.0000000,2022-09-09 13:36:28.0000000,2022-07-07 15:30:53.0000000,2022-09-09 13:36:28.0000000,Hitting Building,Overgrown Tree/Branches,NaN,NaN,40.826875,-73.846715,98.00,Castle Hill-Unionport,11611.0,4B8D16C7-CA9C-49A8-9219-172667426BEA,Level 2,Closed,Good,Full,14.0,57CF781A-0975-4BFE-9904-D6CD8A6EE5A3,2022-09-09 13:36:22.0000000,2022-09-09 13:36:22.0000000,2022-09-09 13:36:22.0000000,NaN,NaN,NaN,POINT (-73.84681453739462 40.826964362109955),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Plant Tree,Street Tree,Routine,3-1-1 Call Center,Open,Reviewed - Inspection Assigned,Staten Island,502.0,NaN,77D798B5-4687-4430-A2AE-AE693E674AB0,2021-04-13 18:09:06.0000000,NaN,2022-07-07 15:28:00.0000000,2022-10-03 13:04:00.0000000,For One Address,New Tree Request,NaN,Business\n\nBusiness,40.599059,-74.126842,187.03,Todt Hill-Emerson Hill-Lighthouse Hill-Manor H...,10700.0,77D798B5-4687-4430-A2AE-AE693E674AB0,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hazard,Hanging Limb Other,Routine,3-1-1 Call Center,Open,Work Priority Category - C,Queens,407.0,799570A8-71A4-4361-A6A8-464FA8AD9448,591583A7-B107-49EC-BFA7-415BE6D3EA4B,2021-07-31 20:32:57.0000000,NaN,2022-07-07 15:30:49.0000000,2022-07-08 14:13:46.0000000,Branch Cracked and Will Fall,Damaged Tree,NaN,NaN,40.769165,-73.800688,1147.00,Murray Hill-Broadway Flushing,14193.0,799570A8-71A4-4361-A6A8-464FA8AD9448,Level 2,Closed,Good,Full,33.0,BD916064-C30D-48F7-9397-F66DCF01203D,2022-06-13 04:00:00.0000000,2022-06-13 19:12:00.0000000,2022-06-13 19:09:00.0000000,2022-06-13 19:15:11.0000000,NaN,NaN,POINT (-73.80039800602839 40.7690878004193),Hanging Limb,Pending,8.0,Queens,53D63DEA-0EFE-4DB7-89CB-EE8C5CBDD728,NaN,NaN,NaN,2022-06-13 19:15:00.0000000,2022-06-13 19:15:13.0000000,Con-Ed,NaN,NaN,Risk Mitigation,NaN,POINT (-73.80039800602839 40.7690878004193)
4,Root/Sewer/Sidewalk,Trees and Sidewalks,Routine,3-1-1 Call Center,Closed,T/S Inspected - Repair Pending,Brooklyn,317.0,NaN,3831B9B0-6787-4A4B-90EF-BC83CDB67735,2021-10-14 22:50:02.0000000,2022-08-05 15:40:29.0000000,2022-07-07 15:30:52.0000000,2022-08-05 15:40:29.0000000,Trees and Sidewalks Program,Root/Sewer/Sidewalk Condition,NaN,NaN,40.638333,-73.934780,836.00,East Flatbush-Farragut,16866.0,3831B9B0-6787-4A4B-90EF-BC83CDB67735,Sidewalk Rating,Closed,Good,Full,22.0,1D67BA0C-DDA6-40F2-B4BA-89F6D185AEF6,2022-08-05 15:40:07.0000000,2022-08-05 15:40:07.0000000,2022-08-05 15:40:07.0000000,NaN,NaN,NaN,POINT (-73.93487015325061 40.63845723855442),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge Risk Assessments

In [27]:
# merge SRs + Insps + WOs w/ Risk Assessmts
mergeddf = pd.merge(mergeddf, FRA, on='InsGlobalID', how='left', suffixes=('', '_RA'))
FRA.shape, mergeddf.shape

((174537, 11), (107081, 63))

In [ ]:
# drop cols with all null values, if any
# mergeddf = mergeddf.dropna(axis=1, how='all')

In [28]:
mergeddf.columns.values # what we're working with

array(['SRCategory', 'SRType', 'SRPriority', 'SRSource', 'SRStatus',
       'SRResolution', 'BoroughCode', 'CommunityBoard',
       'ServiceRequestParentGlobalID', 'GlobalID', 'InitiatedDate',
       'SRClosedDate', 'SRCreatedDate', 'SRUpdatedDate', 'Descriptor1',
       'ComplaintType', 'CallerZipCode', 'SRCallerType', 'Latitude',
       'Longitude', 'Census Tract', 'NTA', 'Zip Codes 2', 'SRGlobalID',
       'InspectionType', 'InspectionStatus', 'InspectionTPCondition',
       'InspectionTPStructure', 'TreePointDBH', 'InsGlobalID',
       'InspectionDate', 'InsClosedDate', 'InsCreatedDate',
       'InsUpdatedDate', 'ParentInspectionGlobalID', 'ReinspectionDate',
       'Location', 'WOType', 'WOStatus', 'WOPriority', 'Borough',
       'WOGlobalID', 'WOClosedDate', 'CancelDate', 'CancelReason',
       'WOCreatedDate', 'WOUpdatedDate', 'WOEntity', 'PROJSTARTDATE',
       'WOProject', 'WOCategory', 'RecommendedSpecies', 'Location_WO',
       'RADefect', 'RADefectLocation', 'Failure', 'Imp

# Export joint table

In [ ]:
compression_opts = dict(method='zip', archive_name='joint_public_data_2022.csv')  
mergeddf.to_csv('joint_public_data_2022.zip', index=False, compression=compression_opts) 

# Export to Geojson

In [ ]:
#Sample Format
{
  "features": [
    {
      "type": "Feature",
      "properties": {
        "title": "Lincoln Park",
        "description": "A northside park that is home to the Lincoln Park Zoo"
      },
      "geometry": {
        "coordinates": [-87.637596, 41.940403],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {
        "title": "Burnham Park",
        "description": "A lakefront park on Chicago's south side"
      },
      "geometry": {
        "coordinates": [-87.603735, 41.829985],
        "type": "Point"
      }
    },
  ],
  "type": "FeatureCollection"
}

In [ ]:
# adapted from https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'features':[], 'type':'FeatureCollection'}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'coordinates':[],
                               'type': 'Point'}
                  }

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [ ]:
mergeddf.columns

In [ ]:
mergeddf["riskrating"]

In [ ]:
cols_to_keep = ['srcategory', 'complainttype', 'srtype', 'descriptor1', 'srpriority', 'srstatus', 'srresolution', 
        'initiateddate', 'SRClosedDate', 'SRCreatedDate', 'SRUpdatedDate', 'SRGlobalID',
        'inspectiontype', 'inspectionstatus', 'inspectiontpcondition', 'inspectiontpstructure', 
        'inspectiondate', 'InsCreatedDate', 'InsUpdatedDate',  'InsClosedDate', 'reinspectiondate', 'parentinspectionglobalid', 'InsGlobalID',
        'WOGlobalID','wotype', 'wostatus', 'wocategory', 'woentity', 'woproject', 'wopriority',
        'WOCreatedDate', 'WOUpdatedDate',  'WOClosedDate', 'cancelreason', 'canceldate', 'projstartdate',
        'radefect', 'radefectlocation', 'failure', 'failureimpact', 'impacttarget', 'consequence', 'riskrating', 
        'RAGlobalID', 'RACreatedDate', 'nta', 'boroughcode', 'communityboard', 'zipcode', 'census_tract', 'latitude_SR', 'longitude_SR'] #, 'geometry_SR']

In [ ]:
mergeddf.columns

In [ ]:
mergeddf = mergeddf[cols_to_keep]

In [ ]:
#drop rows with missing values
# df = mergeddf.dropna()

# Data Aggregation

## Columns Inspection

In [ ]:
#data are based on those with geographic data
#sr category
mergeddf.srcategory.unique()

In [ ]:
#complait type
mergeddf.complainttype.unique()

In [ ]:
#sr type
mergeddf.srtype.unique()

In [ ]:
mergeddf.srresolution.unique()

In [ ]:
mergeddf.InsCreatedDate.unique()

In [ ]:
mergeddf.inspectiontype.unique()

In [ ]:
len(mergeddf)

### Data Insights

In [ ]:
len(mergeddf[(mergeddf.InsCreatedDate != "N/A") & (mergeddf.inspectiontype == "N/A")])

In [ ]:
mergeddf.columns

## Add columns for graph & aggregation

In [ ]:
mergeddf.InsGlobalID.unique()

In [ ]:
mergeddf['inspection_attached'] = np.where(mergeddf['InsGlobalID']!= '0', True, False)

In [ ]:
mergeddf['wo_attached'] = np.where(mergeddf['WOGlobalID'] != 'N/A', True, False)

In [ ]:
mergeddf['initiateddate'] = pd.to_datetime(mergeddf['initiateddate'],errors='coerce')
mergeddf['initiated_year'] = pd.DatetimeIndex(mergeddf['initiateddate']).year
mergeddf['initiated_month'] = pd.DatetimeIndex(mergeddf['initiateddate']).month
mergeddf['SRCreatedDate_datetime'] = pd.to_datetime(mergeddf['SRCreatedDate'],errors='coerce')

## Calculate Numbers

### SR Total

In [ ]:
#for global
len(mergeddf)

In [ ]:
#for each month
count_by_month = mergeddf['SRCreatedDate_datetime'].apply(lambda row: row.month)
count_by_month = count_by_month.to_frame()
count_by_month.rename(columns={'SRCreatedDate_datetime' : 'SRCreated_month'}, inplace = "True")

In [ ]:
count_by_month.groupby(["SRCreated_month"]).size()

### Uninspected Service Requests

In [ ]:
uninspected = mergeddf[mergeddf["inspection_attached"] == False]

In [ ]:
#For global
len(uninspected)

In [ ]:
#For each month
uninspected_by_month = uninspected['SRCreatedDate_datetime'].apply(lambda row: row.month).to_frame()
uninspected_by_month.rename(columns={'SRCreatedDate_datetime' : 'SRCreated_month'}, inplace = "True")


In [ ]:
uninspected_by_month.groupby(["SRCreated_month"]).size()

# Dump data to geojson

In [ ]:
#drop rows is there's no geometry info
mergeddf = mergeddf[mergeddf['latitude_SR'].notna()]

In [ ]:
mergeddf['complainttype'].fillna('None Selected', inplace=True)
mergeddf['srtype'].fillna('None Selected', inplace=True)
mergeddf['riskrating'].fillna(0, inplace=True)
mergeddf.fillna('N/A', inplace = True)

In [ ]:
geojson = df_to_geojson(mergeddf, cols_to_keep, lat='latitude_SR', lon='longitude_SR')

In [ ]:
fileName = "{}{}".format(last_fetch_time, ".geojson")
with open(fileName, 'w') as outfile:
    dump(geojson, outfile)